In [60]:
# 1. Read Dataset
# 2. Extract valuable features
# 3. Data preprocessing
#   3.1 Remove strings from data
#   3.2 Informations are equal to each other
# 4. Divide dataset to train and test set
# 5. Create Neural Network
# 6. Compile and Fit
# 7. Let's predict
# 8. Congrats!

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

import torch
from torch import nn
from torch.nn import functional as F